In [130]:
#logika handling but only from key arrows, collision, start, finish done
import pygame
import sys

# Inisialisasi Pygame
pygame.init()

# --- Konfigurasi Utama Window ---
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600

MAZE_IMAGE_PATH = ".//assets/labirin_noBG.png"
MAZE_SCALE_FACTOR = 1.5

# Warna
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
# GREEN dan FINISH_GATE_COLOR tidak diperlukan lagi untuk logika ini
BLUE = (0, 0, 255) # Warna teks menang
UI_BACKGROUND = WHITE

screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Game Labirin - Temukan Jalan Keluar Lain!")

# --- Memuat Gambar Labirin dan Membuat Maze Mask ---
maze_image = None
maze_mask = None
actual_maze_width = 0
actual_maze_height = 0

try:
    maze_image_original = pygame.image.load(MAZE_IMAGE_PATH).convert_alpha()
    original_maze_width = maze_image_original.get_width()
    original_maze_height = maze_image_original.get_height()
    scaled_maze_width = int(original_maze_width * MAZE_SCALE_FACTOR)
    scaled_maze_height = int(original_maze_height * MAZE_SCALE_FACTOR)
    maze_image = pygame.transform.scale(maze_image_original, (scaled_maze_width, scaled_maze_height))
    actual_maze_width = maze_image.get_width()
    actual_maze_height = maze_image.get_height()
    maze_mask_threshold = 120
    maze_mask = pygame.mask.from_surface(maze_image, threshold=maze_mask_threshold)
    print(f"Gambar labirin '{MAZE_IMAGE_PATH}' dimuat, diskalakan menjadi ({actual_maze_width}x{actual_maze_height}).")
    print(f"Maze mask dibuat dengan threshold alpha > {maze_mask_threshold}.")
except pygame.error as e:
    print(f"Tidak dapat memuat gambar labirin '{MAZE_IMAGE_PATH}': {e}")
    actual_maze_width = int((WINDOW_WIDTH // 2) * MAZE_SCALE_FACTOR)
    actual_maze_height = int((WINDOW_HEIGHT // 2) * MAZE_SCALE_FACTOR)
    print("Menggunakan ukuran placeholder.")

MAZE_OFFSET_X = (WINDOW_WIDTH - actual_maze_width) // 2
MAZE_OFFSET_Y = (WINDOW_HEIGHT - actual_maze_height) // 2

# --- Karakter (Kotak) dan Player Mask ---
player_size = 15
player_speed = 2 # Kecepatan pemain (sesuaikan jika perlu)

# Posisi awal pemain berdasarkan rasio yang diberikan
player_start_x_in_maze_ratio = 0.46
player_start_y_in_maze_ratio = 0.00 # Tepat di tepi atas

player_surface = pygame.Surface((player_size, player_size), pygame.SRCALPHA)
player_surface.fill(RED)
player_mask = pygame.mask.from_surface(player_surface)

player_rect = pygame.Rect(
    MAZE_OFFSET_X + int(actual_maze_width * player_start_x_in_maze_ratio),
    MAZE_OFFSET_Y + int(actual_maze_height * player_start_y_in_maze_ratio),
    player_size, player_size
)

# --- Variabel untuk Logika Kemenangan Dinamis ---
initial_player_center_x = player_rect.centerx
initial_player_center_y = player_rect.centery
player_has_moved_from_spawn = False # Awalnya pemain belum dianggap bergerak dari spawn

# Jarak minimum dari titik spawn agar sebuah perimeter terbuka dianggap "baru" atau "tujuan".
# "Sekecilnya agar lebih ketat": kita set misalnya 2x ukuran pemain.
# Artinya, pusat pemain harus bergeser setidaknya sejauh ini dari pusat spawn awalnya.
MIN_DISTANCE_FROM_SPAWN_FOR_WIN = player_size * 2

# --- Hapus `finish_gate_rect` dan terkaitnya ---
# finish_gate_rect = ... (Tidak diperlukan lagi)

# --- Font ---
font = pygame.font.Font(None, 74)
small_font = pygame.font.Font(None, 36)

# --- Fungsi Deteksi Tabrakan Baru menggunakan Masks ---
def check_maze_collision_with_masks(p_rect, p_mask, m_mask, m_offset_x, m_offset_y):
    if not m_mask or not p_mask: # Jika maze_mask atau player_mask tidak ada
        return False
    offset_x = p_rect.x - m_offset_x
    offset_y = p_rect.y - m_offset_y
    collision_point = m_mask.overlap(p_mask, (offset_x, offset_y))
    return True if collision_point else False

# --- Batas Area Labirin ---
maze_boundary_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)

# --- Game Loop ---
running = True
game_won = False
clock = pygame.time.Clock()

print(f"Posisi Awal Pemain (Tengah): ({initial_player_center_x}, {initial_player_center_y})")
print(f"Batas Jarak Minimum untuk Menang dari Spawn: {MIN_DISTANCE_FROM_SPAWN_FOR_WIN}")
print("Kontrol Game: Tombol panah untuk bergerak.")
if not maze_image or not maze_mask:
    print("PERHATIAN: Gambar labirin atau mask labirin tidak dimuat.")

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if not game_won:
        keys = pygame.key.get_pressed()
        original_player_x = player_rect.x
        original_player_y = player_rect.y

        # Gerakan Horizontal
        requested_dx = 0
        if keys[pygame.K_LEFT]: requested_dx -= player_speed
        if keys[pygame.K_RIGHT]: requested_dx += player_speed
        if requested_dx != 0:
            player_rect.x += requested_dx
            if check_maze_collision_with_masks(player_rect, player_mask, maze_mask, MAZE_OFFSET_X, MAZE_OFFSET_Y):
                player_rect.x = original_player_x
        
        # Gerakan Vertikal
        requested_dy = 0
        if keys[pygame.K_UP]: requested_dy -= player_speed
        if keys[pygame.K_DOWN]: requested_dy += player_speed
        if requested_dy != 0:
            player_rect.y += requested_dy
            if check_maze_collision_with_masks(player_rect, player_mask, maze_mask, MAZE_OFFSET_X, MAZE_OFFSET_Y):
                player_rect.y = original_player_y
        
        if actual_maze_width > 0 and actual_maze_height > 0:
             player_rect.clamp_ip(maze_boundary_rect)
        
        # --- Logika untuk menandai bahwa pemain telah bergerak dari spawn awal ---
        if not player_has_moved_from_spawn:
            # Cek apakah posisi tengah pemain sudah bergeser dari posisi tengah awal
            # (misalnya, lebih dari setengah ukuran pemain untuk dianggap "bergerak")
            current_center_vec = pygame.math.Vector2(player_rect.centerx, player_rect.centery)
            initial_center_vec = pygame.math.Vector2(initial_player_center_x, initial_player_center_y)
            if current_center_vec.distance_to(initial_center_vec) > player_size * 0.5: # Toleransi kecil
                player_has_moved_from_spawn = True
                print("Pemain telah bergerak dari posisi spawn awal.")
        
        # --- Logika Kemenangan BARU (berdasarkan aturan Anda) ---
        if player_has_moved_from_spawn: # ATURAN 1: Pemain harus sudah bergerak dari spawn
            # Cek apakah pemain berada di salah satu tepi luar maze_boundary_rect
            on_left_edge = (player_rect.left == maze_boundary_rect.left and player_rect.width > 0)
            on_right_edge = (player_rect.right == maze_boundary_rect.right and player_rect.width > 0)
            on_top_edge = (player_rect.top == maze_boundary_rect.top and player_rect.height > 0)
            on_bottom_edge = (player_rect.bottom == maze_boundary_rect.bottom and player_rect.height > 0)

            is_on_perimeter = on_left_edge or on_right_edge or on_top_edge or on_bottom_edge
            
            if is_on_perimeter:
                # ATURAN 2: Jarak dari titik spawn awal harus signifikan
                current_center_vec = pygame.math.Vector2(player_rect.centerx, player_rect.centery)
                initial_center_vec = pygame.math.Vector2(initial_player_center_x, initial_player_center_y)
                distance_from_initial_spawn = current_center_vec.distance_to(initial_center_vec)

                if distance_from_initial_spawn > MIN_DISTANCE_FROM_SPAWN_FOR_WIN:
                    # Jika pemain di perimeter, sudah bergerak dari spawn, DAN jaraknya cukup dari spawn awal,
                    # maka ini adalah "Area Terbuka 2" yang valid.
                    # Deteksi mask sebelumnya akan mencegah pemain berada di perimeter jika itu dinding solid.
                    game_won = True
                    print(f"MENANG! Mencapai perimeter terbuka lain. Jarak dari spawn: {distance_from_initial_spawn:.1f}")


    # --- Drawing ---
    screen.fill(WHITE)
    pygame.draw.rect(screen, UI_BACKGROUND, (0, 0, WINDOW_WIDTH, MAZE_OFFSET_Y))
    pygame.draw.rect(screen, UI_BACKGROUND, (0, MAZE_OFFSET_Y + actual_maze_height, WINDOW_WIDTH, WINDOW_HEIGHT - (MAZE_OFFSET_Y + actual_maze_height)))
    pygame.draw.rect(screen, UI_BACKGROUND, (0, MAZE_OFFSET_Y, MAZE_OFFSET_X, actual_maze_height))
    pygame.draw.rect(screen, UI_BACKGROUND, (MAZE_OFFSET_X + actual_maze_width, MAZE_OFFSET_Y, WINDOW_WIDTH - (MAZE_OFFSET_X + actual_maze_width), actual_maze_height))

    score_text_surface = small_font.render("Skor: 0", True, BLACK)
    screen.blit(score_text_surface, (10, 10))
    hint_button_width = 100; hint_button_height = 30; hint_button_margin = 10 
    hint_button_rect = pygame.Rect( WINDOW_WIDTH - hint_button_width - hint_button_margin, hint_button_margin, hint_button_width, hint_button_height)
    pygame.draw.rect(screen, (0,200,0), hint_button_rect)
    hint_text_surface = small_font.render("Hint", True, BLACK)
    hint_text_rect = hint_text_surface.get_rect(center=hint_button_rect.center)
    screen.blit(hint_text_surface, hint_text_rect)

    if maze_image:
        screen.blit(maze_image, (MAZE_OFFSET_X, MAZE_OFFSET_Y))
    else:
        if actual_maze_width > 0 and actual_maze_height > 0:
            placeholder_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)
            pygame.draw.rect(screen, BLACK, placeholder_rect, 2)
            text_surface = small_font.render("Placeholder Labirin", True, BLACK)
            screen.blit(text_surface, (placeholder_rect.centerx - text_surface.get_width() // 2, placeholder_rect.centery - text_surface.get_height() // 2))
            
    pygame.draw.rect(screen, RED, player_rect)
    # Tidak ada finish_gate_rect yang perlu digambar lagi

    if game_won:
        win_text_surface = font.render("KAMU MENANG!", True, BLUE)
        win_text_rect = win_text_surface.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
        pygame.draw.rect(screen, WHITE, win_text_rect.inflate(20,20))
        screen.blit(win_text_surface, win_text_rect)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

Gambar labirin './/assets/labirin_noBG.png' dimuat, diskalakan menjadi (483x483).
Maze mask dibuat dengan threshold alpha > 120.
Posisi Awal Pemain (Tengah): (387, 65)
Batas Jarak Minimum untuk Menang dari Spawn: 30
Kontrol Game: Tombol panah untuk bergerak.
Pemain telah bergerak dari posisi spawn awal.


SystemExit: 

In [134]:
import pygame
import sys
import cv2 # Added for OpenCV
import mediapipe as mp # Added for MediaPipe

# Inisialisasi Pygame
pygame.init()

# --- Konfigurasi Utama Window (Pygame) ---
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600

MAZE_IMAGE_PATH = ".//assets/labirin_noBG.png" # Pastikan path ini benar
MAZE_SCALE_FACTOR = 1.5

# Warna
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0) # Warna pemain
BLUE = (0, 0, 255) # Warna teks menang
GREEN_NEUTRAL = (0, 255, 0) # Warna kotak netral
UI_BACKGROUND = WHITE

screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Game Labirin - Kontrol Hidung Dinamis!")

# --- Konfigurasi OpenCV dan MediaPipe ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0) # 0 untuk webcam default
if not cap.isOpened():
    print("Tidak dapat membuka kamera. Pastikan webcam terhubung dan tidak digunakan oleh aplikasi lain.")
    # sys.exit() # Pertimbangkan untuk keluar jika kamera krusial

# --- Variabel untuk Kontrol Hidung Dinamis ---
initial_nose_x = 0
initial_nose_y = 0
initial_nose_position_set = False # Status apakah posisi hidung awal sudah ditentukan
NEUTRAL_ZONE_RADIUS = 35 # Radius di sekitar titik hidung awal untuk zona netral (stop)
                         # Kotak akan berukuran (NEUTRAL_ZONE_RADIUS*2) x (NEUTRAL_ZONE_RADIUS*2)

# --- Memuat Gambar Labirin dan Membuat Maze Mask ---
maze_image = None
maze_mask = None
actual_maze_width = 0
actual_maze_height = 0

try:
    maze_image_original = pygame.image.load(MAZE_IMAGE_PATH).convert_alpha()
    original_maze_width = maze_image_original.get_width()
    original_maze_height = maze_image_original.get_height()
    scaled_maze_width = int(original_maze_width * MAZE_SCALE_FACTOR)
    scaled_maze_height = int(original_maze_height * MAZE_SCALE_FACTOR)
    maze_image = pygame.transform.scale(maze_image_original, (scaled_maze_width, scaled_maze_height))
    actual_maze_width = maze_image.get_width()
    actual_maze_height = maze_image.get_height()
    maze_mask_threshold = 120
    maze_mask = pygame.mask.from_surface(maze_image, threshold=maze_mask_threshold)
    print(f"Gambar labirin '{MAZE_IMAGE_PATH}' dimuat, diskalakan menjadi ({actual_maze_width}x{actual_maze_height}).")
    print(f"Maze mask dibuat dengan threshold alpha > {maze_mask_threshold}.")
except pygame.error as e:
    print(f"Tidak dapat memuat gambar labirin '{MAZE_IMAGE_PATH}': {e}")
    actual_maze_width = int((WINDOW_WIDTH // 2) * MAZE_SCALE_FACTOR)
    actual_maze_height = int((WINDOW_HEIGHT // 2) * MAZE_SCALE_FACTOR)
    print("Menggunakan ukuran placeholder untuk labirin.")

MAZE_OFFSET_X = (WINDOW_WIDTH - actual_maze_width) // 2
MAZE_OFFSET_Y = (WINDOW_HEIGHT - actual_maze_height) // 2

# --- Karakter (Kotak) dan Player Mask ---
player_size = 15
player_speed = 2

player_start_x_in_maze_ratio = 0.46
player_start_y_in_maze_ratio = 0.00

player_surface = pygame.Surface((player_size, player_size), pygame.SRCALPHA)
player_surface.fill(RED)
player_mask = pygame.mask.from_surface(player_surface)

player_rect = pygame.Rect(
    MAZE_OFFSET_X + int(actual_maze_width * player_start_x_in_maze_ratio),
    MAZE_OFFSET_Y + int(actual_maze_height * player_start_y_in_maze_ratio),
    player_size, player_size
)

# --- Variabel untuk Logika Kemenangan Dinamis ---
initial_player_center_x = player_rect.centerx
initial_player_center_y = player_rect.centery
player_has_moved_from_spawn = False
MIN_DISTANCE_FROM_SPAWN_FOR_WIN = player_size * 2

# --- Font ---
font = pygame.font.Font(None, 74)
small_font = pygame.font.Font(None, 36)

# --- Fungsi Deteksi Tabrakan Baru menggunakan Masks ---
def check_maze_collision_with_masks(p_rect, p_mask, m_mask, m_offset_x, m_offset_y):
    if not m_mask or not p_mask:
        return False
    offset_x = p_rect.x - m_offset_x
    offset_y = p_rect.y - m_offset_y
    collision_point = m_mask.overlap(p_mask, (offset_x, offset_y))
    return True if collision_point else False

# --- Batas Area Labirin ---
maze_boundary_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)

# --- Game Loop ---
running = True
game_won = False
clock = pygame.time.Clock()

print(f"Posisi Awal Pemain (Tengah): ({initial_player_center_x}, {initial_player_center_y})")
print(f"Batas Jarak Minimum untuk Menang dari Spawn: {MIN_DISTANCE_FROM_SPAWN_FOR_WIN}")
if not maze_image or not maze_mask:
    print("PERHATIAN: Gambar labirin atau mask labirin tidak dimuat.")
if not cap.isOpened():
    print("PERHATIAN: Kamera tidak dapat diakses. Kontrol hidung tidak akan berfungsi.")
else:
    print("Posisikan hidung Anda di tengah kamera untuk kalibrasi awal.")
    print(f"Zona netral (berhenti) akan sebesar {NEUTRAL_ZONE_RADIUS*2}x{NEUTRAL_ZONE_RADIUS*2} piksel di sekitar hidung awal Anda.")


cam_window_name = 'Kontrol Hidung - Kalibrasi & Kuadran'

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    dx_nose, dy_nose = 0, 0 # Reset pergerakan hidung setiap frame

    if cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Gagal mengambil frame dari kamera.")
        else:
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            cam_h, cam_w, _ = frame.shape
            
            results = face_mesh.process(frame_rgb)

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    nose_tip = face_landmarks.landmark[1]
                    current_nose_x = int(nose_tip.x * cam_w)
                    current_nose_y = int(nose_tip.y * cam_h)

                    # Gambar titik di hidung saat ini
                    cv2.circle(frame, (current_nose_x, current_nose_y), 5, (0, 255, 255), -1) # Kuning

                    if not initial_nose_position_set:
                        initial_nose_x = current_nose_x
                        initial_nose_y = current_nose_y
                        initial_nose_position_set = True
                        print(f"Titik tengah kontrol hidung DIKALIBRASI di: ({initial_nose_x}, {initial_nose_y})")
                        print("Gerakkan hidung keluar dari kotak hijau untuk bergerak.")
                    
                    if initial_nose_position_set:
                        # Gambar kotak netral (zona berhenti)
                        cv2.rectangle(frame,
                                      (initial_nose_x - NEUTRAL_ZONE_RADIUS, initial_nose_y - NEUTRAL_ZONE_RADIUS),
                                      (initial_nose_x + NEUTRAL_ZONE_RADIUS, initial_nose_y + NEUTRAL_ZONE_RADIUS),
                                      GREEN_NEUTRAL, 2)

                        # Gambar garis kuadran relatif terhadap initial_nose_position
                        cv2.line(frame, (initial_nose_x, 0), (initial_nose_x, cam_h), (255, 0, 0), 1)
                        cv2.line(frame, (0, initial_nose_y), (cam_w, initial_nose_y), (255, 0, 0), 1)

                        # Cek apakah hidung berada di dalam zona netral
                        is_in_neutral_zone = (
                            initial_nose_x - NEUTRAL_ZONE_RADIUS < current_nose_x < initial_nose_x + NEUTRAL_ZONE_RADIUS and
                            initial_nose_y - NEUTRAL_ZONE_RADIUS < current_nose_y < initial_nose_y + NEUTRAL_ZONE_RADIUS
                        )

                        if is_in_neutral_zone:
                            dx_nose = 0
                            dy_nose = 0
                        else:
                            # Hidung di luar zona netral, tentukan kuadran relatif terhadap initial_nose_position
                            if current_nose_x > initial_nose_x and current_nose_y < initial_nose_y: # Q1: Kanan Atas
                                dx_nose = player_speed
                                dy_nose = -player_speed
                            elif current_nose_x < initial_nose_x and current_nose_y < initial_nose_y: # Q2: Kiri Atas
                                dx_nose = -player_speed
                                dy_nose = -player_speed
                            elif current_nose_x < initial_nose_x and current_nose_y > initial_nose_y: # Q3: Kiri Bawah
                                dx_nose = -player_speed
                                dy_nose = player_speed
                            elif current_nose_x > initial_nose_x and current_nose_y > initial_nose_y: # Q4: Kanan Bawah
                                dx_nose = player_speed
                                dy_nose = player_speed
                    break 
            else: # Jika tidak ada wajah terdeteksi setelah kalibrasi, pemain berhenti
                if initial_nose_position_set:
                    dx_nose = 0
                    dy_nose = 0
                    print("Wajah tidak terdeteksi, pemain berhenti.") # Bisa di-komen jika terlalu berisik
                else:
                    # Pesan untuk kalibrasi awal jika belum
                     cv2.putText(frame, "Posisikan hidung untuk kalibrasi", (cam_w // 2 - 200, cam_h // 2),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)


            cv2.imshow(cam_window_name, frame)

    # --- Logika Game ---
    if not game_won:
        original_player_x = player_rect.x
        original_player_y = player_rect.y

        if dx_nose != 0:
            player_rect.x += dx_nose
            if check_maze_collision_with_masks(player_rect, player_mask, maze_mask, MAZE_OFFSET_X, MAZE_OFFSET_Y):
                player_rect.x = original_player_x
        
        if dy_nose != 0:
            player_rect.y += dy_nose
            if check_maze_collision_with_masks(player_rect, player_mask, maze_mask, MAZE_OFFSET_X, MAZE_OFFSET_Y):
                player_rect.y = original_player_y
        
        if actual_maze_width > 0 and actual_maze_height > 0:
            player_rect.clamp_ip(maze_boundary_rect)
        
        if not player_has_moved_from_spawn:
            current_center_vec = pygame.math.Vector2(player_rect.centerx, player_rect.centery)
            initial_center_vec = pygame.math.Vector2(initial_player_center_x, initial_player_center_y)
            if current_center_vec.distance_to(initial_center_vec) > player_size * 0.5:
                player_has_moved_from_spawn = True
                print("Pemain telah bergerak dari posisi spawn awal.")
        
        if player_has_moved_from_spawn:
            on_left_edge = (player_rect.left == maze_boundary_rect.left and player_rect.width > 0)
            on_right_edge = (player_rect.right == maze_boundary_rect.right and player_rect.width > 0)
            on_top_edge = (player_rect.top == maze_boundary_rect.top and player_rect.height > 0)
            on_bottom_edge = (player_rect.bottom == maze_boundary_rect.bottom and player_rect.height > 0)
            is_on_perimeter = on_left_edge or on_right_edge or on_top_edge or on_bottom_edge
            
            if is_on_perimeter:
                current_center_vec = pygame.math.Vector2(player_rect.centerx, player_rect.centery)
                initial_center_vec = pygame.math.Vector2(initial_player_center_x, initial_player_center_y)
                distance_from_initial_spawn = current_center_vec.distance_to(initial_center_vec)
                if distance_from_initial_spawn > MIN_DISTANCE_FROM_SPAWN_FOR_WIN:
                    game_won = True
                    print(f"MENANG! Mencapai perimeter terbuka lain. Jarak dari spawn: {distance_from_initial_spawn:.1f}")

    # --- Drawing (Pygame) ---
    screen.fill(WHITE)
    pygame.draw.rect(screen, UI_BACKGROUND, (0, 0, WINDOW_WIDTH, MAZE_OFFSET_Y))
    pygame.draw.rect(screen, UI_BACKGROUND, (0, MAZE_OFFSET_Y + actual_maze_height, WINDOW_WIDTH, WINDOW_HEIGHT - (MAZE_OFFSET_Y + actual_maze_height)))
    pygame.draw.rect(screen, UI_BACKGROUND, (0, MAZE_OFFSET_Y, MAZE_OFFSET_X, actual_maze_height))
    pygame.draw.rect(screen, UI_BACKGROUND, (MAZE_OFFSET_X + actual_maze_width, MAZE_OFFSET_Y, WINDOW_WIDTH - (MAZE_OFFSET_X + actual_maze_width), actual_maze_height))

    score_text_surface = small_font.render("Skor: 0", True, BLACK)
    screen.blit(score_text_surface, (10, 10))
    hint_button_width = 100; hint_button_height = 30; hint_button_margin = 10 
    hint_button_rect = pygame.Rect( WINDOW_WIDTH - hint_button_width - hint_button_margin, hint_button_margin, hint_button_width, hint_button_height)
    pygame.draw.rect(screen, (0,200,0), hint_button_rect)
    hint_text_surface = small_font.render("Hint", True, BLACK)
    hint_text_rect = hint_text_surface.get_rect(center=hint_button_rect.center)
    screen.blit(hint_text_surface, hint_text_rect)

    if maze_image:
        screen.blit(maze_image, (MAZE_OFFSET_X, MAZE_OFFSET_Y))
    else:
        if actual_maze_width > 0 and actual_maze_height > 0:
            placeholder_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)
            pygame.draw.rect(screen, BLACK, placeholder_rect, 2)
            text_surface = small_font.render("Placeholder Labirin", True, BLACK)
            screen.blit(text_surface, (placeholder_rect.centerx - text_surface.get_width() // 2, placeholder_rect.centery - text_surface.get_height() // 2))
            
    pygame.draw.rect(screen, RED, player_rect)

    if game_won:
        win_text_surface = font.render("KAMU MENANG!", True, BLUE)
        win_text_rect = win_text_surface.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
        pygame.draw.rect(screen, WHITE, win_text_rect.inflate(20,20))
        screen.blit(win_text_surface, win_text_rect)

    pygame.display.flip()
    
    if cv2.waitKey(1) & 0xFF == 27:
        running = False
        
    clock.tick(30)

# --- Cleanup ---
if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()
pygame.quit()
sys.exit()

Gambar labirin './/assets/labirin_noBG.png' dimuat, diskalakan menjadi (483x483).
Maze mask dibuat dengan threshold alpha > 120.
Posisi Awal Pemain (Tengah): (387, 65)
Batas Jarak Minimum untuk Menang dari Spawn: 30
Posisikan hidung Anda di tengah kamera untuk kalibrasi awal.
Zona netral (berhenti) akan sebesar 70x70 piksel di sekitar hidung awal Anda.
Titik tengah kontrol hidung DIKALIBRASI di: (315, 311)
Gerakkan hidung keluar dari kotak hijau untuk bergerak.
Pemain telah bergerak dari posisi spawn awal.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
Wajah tidak terdeteksi, pemain berhenti.
MENANG! Mencapai perimeter terbuka la

SystemExit: 

In [1]:
import pygame
import sys
import cv2
import mediapipe as mp
import speech_recognition as sr
import threading
import os

pygame.init()

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600

BASE_ASSET_PATH = ".//assets"
MAZE_IMAGE_PATH_NO_HINT = os.path.join(BASE_ASSET_PATH, "labirin_noBG.png")
MAZE_IMAGE_PATH_HINT = os.path.join(BASE_ASSET_PATH, "hint_labirin.png")
# Path labirin yang sedang ditampilkan
CURRENT_DISPLAY_MAZE_PATH = MAZE_IMAGE_PATH_NO_HINT # Dimulai dengan tanpa hint

MAZE_SCALE_FACTOR = 1.5
WHITE, BLACK, RED, BLUE = (255,255,255), (0,0,0), (255,0,0), (0,0,255)
GREEN_NEUTRAL, UI_BACKGROUND, YELLOW_HINT_MSG = (0,255,0), WHITE, (200,200,0)

screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Game Labirin - Kontrol Hidung & Suara!")

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)
if not cap.isOpened(): print("Tidak dapat membuka kamera.")

initial_nose_x, initial_nose_y = 0, 0
initial_nose_position_set = False
NEUTRAL_ZONE_RADIUS = 35

recognizer = sr.Recognizer()
microphone = None
try:
    microphone = sr.Microphone()
except Exception as e:
    print(f"Error inisialisasi microphone: {e}. Speech recognition tidak akan berfungsi.")

hint_requested_by_voice, hint_active = False, False
listening_status_message = ""

# --- Variabel Global untuk Aset Labirin ---
maze_display_image = None       # Surface Pygame untuk apa yang ditampilkan di layar
maze_collision_mask = None      # Mask Pygame HANYA untuk deteksi tabrakan (dari labirin no-hint)
actual_maze_width, actual_maze_height = 0, 0
MAZE_OFFSET_X, MAZE_OFFSET_Y = 0, 0
maze_boundary_rect = None # Rect untuk batas luar labirin yang ditampilkan


# --- Fungsi untuk Memuat Aset Labirin ---
def load_maze_assets(image_path_to_load, create_collision_mask_from_this_image):
    global maze_display_image, maze_collision_mask, actual_maze_width, actual_maze_height
    global MAZE_OFFSET_X, MAZE_OFFSET_Y, maze_boundary_rect, CURRENT_DISPLAY_MAZE_PATH

    CURRENT_DISPLAY_MAZE_PATH = image_path_to_load
    try:
        loaded_surface = pygame.image.load(image_path_to_load).convert_alpha()
        original_width = loaded_surface.get_width()
        original_height = loaded_surface.get_height()

        scaled_width = int(original_width * MAZE_SCALE_FACTOR)
        scaled_height = int(original_height * MAZE_SCALE_FACTOR)

        maze_display_image = pygame.transform.scale(loaded_surface, (scaled_width, scaled_height))
        actual_maze_width = maze_display_image.get_width()
        actual_maze_height = maze_display_image.get_height()

        MAZE_OFFSET_X = (WINDOW_WIDTH - actual_maze_width) // 2
        MAZE_OFFSET_Y = (WINDOW_HEIGHT - actual_maze_height) // 2
        
        if actual_maze_width > 0 and actual_maze_height > 0:
            maze_boundary_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)
        else: 
            maze_boundary_rect = pygame.Rect(0,0,0,0)

        if create_collision_mask_from_this_image:
            maze_collision_mask = pygame.mask.from_surface(maze_display_image, threshold=120)
            print(f"Masker tabrakan diperbarui dari '{image_path_to_load}'.")
        
        print(f"Gambar '{image_path_to_load}' (untuk tampilan) berhasil dimuat.")
        return True
        
    except pygame.error as e:
        print(f"Tidak dapat memuat gambar labirin '{image_path_to_load}': {e}")
        maze_display_image = pygame.Surface((100,100)) 
        maze_display_image.fill(BLACK) 
        actual_maze_width = 100; actual_maze_height = 100
        MAZE_OFFSET_X = (WINDOW_WIDTH - actual_maze_width) // 2
        MAZE_OFFSET_Y = (WINDOW_HEIGHT - actual_maze_height) // 2
        maze_boundary_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)
        if create_collision_mask_from_this_image: 
             maze_collision_mask = None 
        return False

player_size, player_speed = 15, 2
player_start_x_ratio, player_start_y_ratio = 0.46, 0.00
player_surface = pygame.Surface((player_size, player_size), pygame.SRCALPHA)
player_surface.fill(RED)
player_mask = pygame.mask.from_surface(player_surface)

if not load_maze_assets(MAZE_IMAGE_PATH_NO_HINT, True): 
    print("KRITIKAL: Gagal memuat labirin awal dan maskernya. Game mungkin tidak berfungsi dengan benar.")

player_rect = pygame.Rect(
    MAZE_OFFSET_X + int(actual_maze_width * player_start_x_ratio),
    MAZE_OFFSET_Y + int(actual_maze_height * player_start_y_ratio),
    player_size, player_size
)

initial_player_center_x, initial_player_center_y = player_rect.centerx, player_rect.centery
player_has_moved_from_spawn = False
MIN_DISTANCE_FROM_SPAWN_FOR_WIN = player_size * 2

font, small_font, micro_font = pygame.font.Font(None, 74), pygame.font.Font(None, 36), pygame.font.Font(None, 24)

def check_maze_collision_with_masks(p_rect, p_mask, m_collision_mask, m_offset_x, m_offset_y):
    if not m_collision_mask or not p_mask:
        return False
    offset_x = p_rect.x - m_offset_x
    offset_y = p_rect.y - m_offset_y
    collision_point = m_collision_mask.overlap(p_mask, (offset_x, offset_y))
    return True if collision_point else False

def listen_for_hint_command():
    global hint_requested_by_voice, listening_status_message, running
    if not microphone: 
        listening_status_message = "Mikrofon tidak tersedia."
        return

    print("Thread speech recognition dimulai...")
    try:
        with microphone as source:
            print("Kalibrasi suara sekitar (1 detik)...")
            recognizer.adjust_for_ambient_noise(source, duration=1)
            print("Kalibrasi selesai. Katakan 'dit tolongin dit'.")
            listening_status_message = "Katakan 'dit tolongin dit' untuk hint."
    except Exception as e:
        print(f"Error kalibrasi mic: {e}")
        listening_status_message = "Error kalibrasi mic."

    while running:
        if not microphone: 
            listening_status_message = "Mikrofon error."
            pygame.time.wait(1000)
            continue
        if hint_active:
            listening_status_message = "Hint sudah aktif."
            pygame.time.wait(1000)
            continue

        current_listening_msg = "Mendengarkan..."
        try:
            if not "Mendengarkan..." in listening_status_message and not "Hint diminta" in listening_status_message:
                 listening_status_message = current_listening_msg
            with microphone as source:
                audio = recognizer.listen(source, timeout=3, phrase_time_limit=4)
            
            listening_status_message = "Memproses suara..."
            command = recognizer.recognize_google(audio, language="id-ID").lower()
            print(f"Anda mengatakan: {command}")
            listening_status_message = f"Terdeteksi: {command[:30]}..."

            if "dit tolongin dit" in command and not hint_active:
                print("Perintah hint terdeteksi!")
                hint_requested_by_voice = True
                listening_status_message = "Hint diminta! Mengaktifkan..."
            pygame.time.wait(200)
        except sr.WaitTimeoutError: 
            if not "Katakan" in listening_status_message : listening_status_message = "Katakan 'dit tolongin dit'..."
        except sr.UnknownValueError: 
            if not "Tidak dapat mengenali" in listening_status_message : listening_status_message = "Tidak dapat mengenali suara."
        except sr.RequestError as e: 
            print(f"API Error: {e}"); listening_status_message = "Error koneksi speech API."
        except Exception as e: 
            print(f"Error speech: {e}"); listening_status_message = "Error speech tidak diketahui."
        if not running: break
    print("Thread speech recognition dihentikan.")

running = True
game_won = False
clock = pygame.time.Clock()

if not maze_display_image or not maze_collision_mask:
    print("PERHATIAN: Labirin atau masker tabrakan tidak dimuat dengan benar.")

if microphone:
    listener_thread = threading.Thread(target=listen_for_hint_command, daemon=True)
    listener_thread.start()
else:
    listening_status_message = "Speech recognition tidak aktif (mikrofon error)."

cam_window_name = 'Kontrol Hidung - Kalibrasi & Kuadran'

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if hint_requested_by_voice and not hint_active:
        print("Mengaktifkan hint...")
        if load_maze_assets(MAZE_IMAGE_PATH_HINT, False): 
            hint_active = True
            # !!! BARIS UNTUK MERESET POSISI PLAYER DIHAPUS/DIKOMENTARI !!!
            # player_rect.x = MAZE_OFFSET_X + int(actual_maze_width * player_start_x_ratio)
            # player_rect.y = MAZE_OFFSET_Y + int(actual_maze_height * player_start_y_ratio) + player_size // 3 
            # if maze_boundary_rect: player_rect.clamp_ip(maze_boundary_rect)
            
            print(f"Labirin hint dimuat untuk TAMPILAN. Posisi pemain TETAP. Masker tabrakan dari labirin asli.") # Pesan diubah
            listening_status_message = "Hint sudah aktif."
        else:
            print("Gagal memuat labirin hint.")
            listening_status_message = "Gagal load hint."
        hint_requested_by_voice = False

    dx_nose, dy_nose = 0, 0
    if cap.isOpened():
        success, frame = cap.read()
        if success:
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            cam_h, cam_w, _ = frame.shape
            results = face_mesh.process(frame_rgb)
            if results.multi_face_landmarks:
                 for face_landmarks in results.multi_face_landmarks:
                    nose_tip = face_landmarks.landmark[1]
                    current_nose_x = int(nose_tip.x * cam_w)
                    current_nose_y = int(nose_tip.y * cam_h)
                    cv2.circle(frame, (current_nose_x, current_nose_y), 5, (0, 255, 255), -1)
                    if not initial_nose_position_set:
                        initial_nose_x, initial_nose_y = current_nose_x, current_nose_y
                        initial_nose_position_set = True; print(f"Kalibrasi hidung di: ({initial_nose_x}, {initial_nose_y})")
                    if initial_nose_position_set:
                        cv2.rectangle(frame, (initial_nose_x - NEUTRAL_ZONE_RADIUS, initial_nose_y - NEUTRAL_ZONE_RADIUS), (initial_nose_x + NEUTRAL_ZONE_RADIUS, initial_nose_y + NEUTRAL_ZONE_RADIUS), GREEN_NEUTRAL, 2)
                        cv2.line(frame, (initial_nose_x, 0), (initial_nose_x, cam_h), (255,0,0),1); cv2.line(frame, (0, initial_nose_y), (cam_w, initial_nose_y), (255,0,0),1)
                        is_in_neutral = (initial_nose_x - NEUTRAL_ZONE_RADIUS < current_nose_x < initial_nose_x + NEUTRAL_ZONE_RADIUS and \
                                         initial_nose_y - NEUTRAL_ZONE_RADIUS < current_nose_y < initial_nose_y + NEUTRAL_ZONE_RADIUS)
                        if is_in_neutral: dx_nose, dy_nose = 0,0
                        else:
                            if current_nose_x > initial_nose_x and current_nose_y < initial_nose_y: dx_nose, dy_nose = player_speed, -player_speed
                            elif current_nose_x < initial_nose_x and current_nose_y < initial_nose_y: dx_nose, dy_nose = -player_speed, -player_speed
                            elif current_nose_x < initial_nose_x and current_nose_y > initial_nose_y: dx_nose, dy_nose = -player_speed, player_speed
                            elif current_nose_x > initial_nose_x and current_nose_y > initial_nose_y: dx_nose, dy_nose = player_speed, player_speed
                    break
            else: 
                if initial_nose_position_set: dx_nose, dy_nose = 0,0
                else: cv2.putText(frame, "Posisikan hidung", (cam_w//2-100, cam_h//2), cv2.FONT_HERSHEY_SIMPLEX, 0.8, WHITE, 2)
            cv2.imshow(cam_window_name, frame)

    if not game_won:
        original_player_x, original_player_y = player_rect.x, player_rect.y
        if dx_nose != 0:
            player_rect.x += dx_nose
            if maze_collision_mask and check_maze_collision_with_masks(player_rect, player_mask, maze_collision_mask, MAZE_OFFSET_X, MAZE_OFFSET_Y):
                player_rect.x = original_player_x
        if dy_nose != 0:
            player_rect.y += dy_nose
            if maze_collision_mask and check_maze_collision_with_masks(player_rect, player_mask, maze_collision_mask, MAZE_OFFSET_X, MAZE_OFFSET_Y):
                player_rect.y = original_player_y
        
        if maze_boundary_rect : player_rect.clamp_ip(maze_boundary_rect)
        
        if not player_has_moved_from_spawn:
            current_center_vec = pygame.math.Vector2(player_rect.centerx, player_rect.centery)
            initial_center_vec = pygame.math.Vector2(initial_player_center_x, initial_player_center_y)
            if current_center_vec.distance_to(initial_center_vec) > player_size * 0.5:
                player_has_moved_from_spawn = True; print("Pemain bergerak dari spawn awal.")
        
        if player_has_moved_from_spawn and maze_boundary_rect: 
            on_perimeter = (player_rect.left == maze_boundary_rect.left or \
                            player_rect.right == maze_boundary_rect.right or \
                            player_rect.top == maze_boundary_rect.top or \
                            player_rect.bottom == maze_boundary_rect.bottom) and \
                           (player_rect.width > 0 and player_rect.height > 0) 
            
            if on_perimeter:
                dist_vec = pygame.math.Vector2(player_rect.centerx, player_rect.centery).distance_to(pygame.math.Vector2(initial_player_center_x, initial_player_center_y))
                if dist_vec > MIN_DISTANCE_FROM_SPAWN_FOR_WIN:
                    game_won = True; print(f"MENANG! Jarak dari spawn: {dist_vec:.1f}")

    screen.fill(WHITE)
    pygame.draw.rect(screen, UI_BACKGROUND, (0, 0, WINDOW_WIDTH, MAZE_OFFSET_Y)) 
    pygame.draw.rect(screen, UI_BACKGROUND, (0, MAZE_OFFSET_Y + actual_maze_height, WINDOW_WIDTH, WINDOW_HEIGHT - (MAZE_OFFSET_Y + actual_maze_height))) 
    pygame.draw.rect(screen, UI_BACKGROUND, (0, MAZE_OFFSET_Y, MAZE_OFFSET_X, actual_maze_height)) 
    pygame.draw.rect(screen, UI_BACKGROUND, (MAZE_OFFSET_X + actual_maze_width, MAZE_OFFSET_Y, WINDOW_WIDTH - (MAZE_OFFSET_X + actual_maze_width), actual_maze_height)) 

    score_text_surface = small_font.render("Skor: 0", True, BLACK)
    screen.blit(score_text_surface, (10, 10))
    
    speech_status_surf = micro_font.render(listening_status_message, True, YELLOW_HINT_MSG if "hint" in listening_status_message.lower() or "diminta" in listening_status_message.lower() else BLACK)
    screen.blit(speech_status_surf, (10, WINDOW_HEIGHT - 30))

    hint_btn_rect = pygame.Rect(WINDOW_WIDTH - 110, 10, 100, 30)
    pygame.draw.rect(screen, (0,200,0) if not hint_active else (100,100,100), hint_btn_rect)
    hint_txt_surf = small_font.render("Hint" if not hint_active else "Hint ON", True, BLACK)
    screen.blit(hint_txt_surf, hint_txt_surf.get_rect(center=hint_btn_rect.center))

    if maze_display_image:
        screen.blit(maze_display_image, (MAZE_OFFSET_X, MAZE_OFFSET_Y))
    else: 
        if actual_maze_width > 0 and actual_maze_height > 0: 
            placeholder_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)
            pygame.draw.rect(screen, BLACK, placeholder_rect, 2) 
            text_surface = small_font.render("Error Labirin", True, BLACK) 
            screen.blit(text_surface, (placeholder_rect.centerx - text_surface.get_width() // 2, placeholder_rect.centery - text_surface.get_height() // 2))

    pygame.draw.rect(screen, RED, player_rect)

    if game_won:
        win_text_surface = font.render("KAMU MENANG!", True, BLUE)
        win_text_rect = win_text_surface.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
        pygame.draw.rect(screen, WHITE, win_text_rect.inflate(20,20)) 
        screen.blit(win_text_surface, win_text_rect)

    pygame.display.flip()
    key_cv = cv2.waitKey(1)
    if key_cv & 0xFF == 27: running = False
    clock.tick(30)

if cap.isOpened(): cap.release()
cv2.destroyAllWindows()
pygame.quit()
print("Game ditutup.")
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Masker tabrakan diperbarui dari './/assets\labirin_noBG.png'.
Gambar './/assets\labirin_noBG.png' (untuk tampilan) berhasil dimuat.
Thread speech recognition dimulai...
Kalibrasi suara sekitar (1 detik)...
Kalibrasi hidung di: (382, 306)
Pemain bergerak dari spawn awal.
Kalibrasi selesai. Katakan 'dit tolongin dit'.
Anda mengatakan: cara cek cek
Anda mengatakan: nana nana
Anda mengatakan: akan
Anda mengatakan: dit tolongin dit
Perintah hint terdeteksi!
Mengaktifkan hint...
Gambar './/assets\hint_labirin.png' (untuk tampilan) berhasil dimuat.
Labirin hint dimuat untuk TAMPILAN. Posisi pemain TETAP. Masker tabrakan dari labirin asli.
Thread speech recognition dihentikan.
Game ditutup.


SystemExit: 

d:\DTD-Maze\mulmet_env\Lib\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import pygame
import sys
import cv2
import mediapipe as mp
import speech_recognition as sr
import threading
import os
import imageio

pygame.init()

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600

BASE_ASSET_PATH = ".//assets"
MAZE_IMAGE_PATH_NO_HINT = os.path.join(BASE_ASSET_PATH, "labirin_noBG.png")
MAZE_IMAGE_PATH_HINT = os.path.join(BASE_ASSET_PATH, "hint_labirin.png")
CHARACTER_GIF_PATH = os.path.join(BASE_ASSET_PATH, "denis-gerak-noBG.gif")

CURRENT_DISPLAY_MAZE_PATH = MAZE_IMAGE_PATH_NO_HINT

MAZE_SCALE_FACTOR = 1.5
WHITE, BLACK, RED, BLUE = (255,255,255), (0,0,0), (255,0,0), (0,0,255)
GREEN_NEUTRAL, UI_BACKGROUND, YELLOW_HINT_MSG = (0,255,0), WHITE, (200,200,0)

screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Game Labirin - Kontrol Hidung & Suara!")

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)
if not cap.isOpened(): print("Tidak dapat membuka kamera.")

initial_nose_x, initial_nose_y = 0, 0
initial_nose_position_set = False
NEUTRAL_ZONE_RADIUS = 35

recognizer = sr.Recognizer()
microphone = None
try:
    microphone = sr.Microphone()
except Exception as e:
    print(f"Error inisialisasi microphone: {e}. Speech recognition tidak akan berfungsi.")

hint_requested_by_voice, hint_active = False, False
listening_status_message = ""

maze_display_image = None
maze_collision_mask = None
actual_maze_width, actual_maze_height = 0, 0
MAZE_OFFSET_X, MAZE_OFFSET_Y = 0, 0
maze_boundary_rect = None

def load_maze_assets(image_path_to_load, create_collision_mask_from_this_image):
    global maze_display_image, maze_collision_mask, actual_maze_width, actual_maze_height
    global MAZE_OFFSET_X, MAZE_OFFSET_Y, maze_boundary_rect, CURRENT_DISPLAY_MAZE_PATH

    CURRENT_DISPLAY_MAZE_PATH = image_path_to_load
    try:
        loaded_surface = pygame.image.load(image_path_to_load).convert_alpha()
        original_width = loaded_surface.get_width()
        original_height = loaded_surface.get_height()
        scaled_width = int(original_width * MAZE_SCALE_FACTOR)
        scaled_height = int(original_height * MAZE_SCALE_FACTOR)
        maze_display_image = pygame.transform.scale(loaded_surface, (scaled_width, scaled_height))
        actual_maze_width = maze_display_image.get_width()
        actual_maze_height = maze_display_image.get_height()
        MAZE_OFFSET_X = (WINDOW_WIDTH - actual_maze_width) // 2
        MAZE_OFFSET_Y = (WINDOW_HEIGHT - actual_maze_height) // 2
        if actual_maze_width > 0 and actual_maze_height > 0:
            maze_boundary_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)
        else:
            maze_boundary_rect = pygame.Rect(0,0,0,0)
        if create_collision_mask_from_this_image:
            maze_collision_mask = pygame.mask.from_surface(maze_display_image, threshold=120)
            print(f"Masker tabrakan diperbarui dari '{image_path_to_load}'.")
        print(f"Gambar '{image_path_to_load}' (untuk tampilan) berhasil dimuat.")
        return True
    except pygame.error as e:
        print(f"Tidak dapat memuat gambar labirin '{image_path_to_load}': {e}")
        maze_display_image = pygame.Surface((100,100))
        maze_display_image.fill(BLACK)
        actual_maze_width = 100; actual_maze_height = 100
        MAZE_OFFSET_X = (WINDOW_WIDTH - actual_maze_width) // 2
        MAZE_OFFSET_Y = (WINDOW_HEIGHT - actual_maze_height) // 2
        maze_boundary_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)
        if create_collision_mask_from_this_image:
            maze_collision_mask = None
        return False

PLAYER_SCALE = 0.5
player_speed = 2
player_start_x_ratio, player_start_y_ratio = 0.46, 0.00
player_frames = []
current_player_frame_index = 0
last_animation_update_time = 0
ANIMATION_FRAME_DURATION = 100
player_actual_width, player_actual_height = 15, 15
player_mask = None

if not load_maze_assets(MAZE_IMAGE_PATH_NO_HINT, True):
    print("KRITIKAL: Gagal memuat labirin awal dan maskernya.")

try:
    gif_images = imageio.mimread(CHARACTER_GIF_PATH)
    for frame_data in gif_images:
        if frame_data.ndim == 3 and frame_data.shape[0] > 0 and frame_data.shape[1] > 0 : # Pastikan frame valid
            frame_surface = None
            if frame_data.shape[2] == 4: # RGBA
                frame_surface = pygame.image.frombuffer(frame_data.tobytes(), frame_data.shape[:2][::-1], "RGBA").convert_alpha()
            elif frame_data.shape[2] == 3: # RGB
                frame_surface = pygame.image.frombuffer(frame_data.tobytes(), frame_data.shape[:2][::-1], "RGB").convert_alpha()
            
            if frame_surface:
                original_width, original_height = frame_surface.get_size()
                scaled_width = int(original_width * PLAYER_SCALE)
                scaled_height = int(original_height * PLAYER_SCALE)
                if scaled_width > 0 and scaled_height > 0: # Hindari scaling ke nol
                    scaled_frame = pygame.transform.scale(frame_surface, (scaled_width, scaled_height))
                    player_frames.append(scaled_frame)
                else:
                    print(f"Warning: Skipping frame karena ukuran setelah scaling menjadi nol atau negatif ({scaled_width}x{scaled_height}). Original: {original_width}x{original_height}, Scale: {PLAYER_SCALE}")


    if not player_frames:
        raise ValueError("Tidak ada frame valid yang dimuat dari GIF atau semua frame berukuran nol setelah scaling.")

    player_actual_width, player_actual_height = player_frames[0].get_size() # Ukuran dari frame pertama yg di-scale
    player_mask = pygame.mask.from_surface(player_frames[0]) # Mask dari frame pertama yg di-scale
    print(f"Karakter GIF '{CHARACTER_GIF_PATH}' berhasil dimuat ({len(player_frames)} frames). Ukuran setelah skala: {player_actual_width}x{player_actual_height} (Skala: {PLAYER_SCALE})")

except Exception as e:
    print(f"Error memuat karakter GIF '{CHARACTER_GIF_PATH}': {e}")
    print("Fallback: Menggunakan kotak merah sebagai pemain.")
    player_actual_width, player_actual_height = 15, 15
    fallback_surface = pygame.Surface((player_actual_width, player_actual_height), pygame.SRCALPHA)
    fallback_surface.fill(RED)
    if player_frames: # Jika list sudah ada tapi error kemudian
        player_frames.clear() # Kosongkan jika ada frame parsial
    player_frames.append(fallback_surface)
    player_mask = pygame.mask.from_surface(player_frames[0])


player_rect = pygame.Rect(
    MAZE_OFFSET_X + int(actual_maze_width * player_start_x_ratio),
    MAZE_OFFSET_Y + int(actual_maze_height * player_start_y_ratio),
    player_actual_width,
    player_actual_height
)

initial_player_center_x, initial_player_center_y = player_rect.centerx, player_rect.centery
player_has_moved_from_spawn = False
MIN_DISTANCE_FROM_SPAWN_FOR_WIN = max(player_actual_width, player_actual_height) * 1.5

font, small_font, micro_font = pygame.font.Font(None, 74), pygame.font.Font(None, 36), pygame.font.Font(None, 24)

def check_maze_collision_with_masks(p_rect, p_mask, m_collision_mask, m_offset_x, m_offset_y):
    if not m_collision_mask or not p_mask:
        return False
    offset_x = p_rect.x - m_offset_x
    offset_y = p_rect.y - m_offset_y
    collision_point = m_collision_mask.overlap(p_mask, (offset_x, offset_y))
    return True if collision_point else False

def listen_for_hint_command():
    global hint_requested_by_voice, listening_status_message, running
    if not microphone:
        listening_status_message = "Mikrofon tidak tersedia."
        return
    print("Thread speech recognition dimulai...")
    try:
        with microphone as source:
            print("Kalibrasi suara sekitar (1 detik)...")
            recognizer.adjust_for_ambient_noise(source, duration=1)
            print("Kalibrasi selesai. Katakan 'dit tolongin dit'.")
            listening_status_message = "Katakan 'dit tolongin dit' untuk hint."
    except Exception as e:
        print(f"Error kalibrasi mic: {e}")
        listening_status_message = "Error kalibrasi mic."
    while running:
        if not microphone:
            listening_status_message = "Mikrofon error."
            pygame.time.wait(1000); continue
        if hint_active:
            listening_status_message = "Hint sudah aktif."
            pygame.time.wait(1000); continue
        current_listening_msg = "Mendengarkan..."
        try:
            if not ("Mendengarkan..." in listening_status_message or "Hint diminta" in listening_status_message or "Terdeteksi" in listening_status_message):
                listening_status_message = current_listening_msg
            with microphone as source:
                audio = recognizer.listen(source, timeout=3, phrase_time_limit=4)
            listening_status_message = "Memproses suara..."
            command = recognizer.recognize_google(audio, language="id-ID").lower()
            print(f"Anda mengatakan: {command}")
            listening_status_message = f"Terdeteksi: {command[:30]}..."
            if "dit tolongin dit" in command and not hint_active:
                print("Perintah hint terdeteksi!")
                hint_requested_by_voice = True
                listening_status_message = "Hint diminta! Mengaktifkan..."
            pygame.time.wait(200)
        except sr.WaitTimeoutError:
            if not "Katakan" in listening_status_message: listening_status_message = "Katakan 'dit tolongin dit'..."
        except sr.UnknownValueError:
            if not "Tidak dapat mengenali" in listening_status_message: listening_status_message = "Tidak dapat mengenali suara."
        except sr.RequestError as e:
            print(f"API Error: {e}"); listening_status_message = "Error koneksi speech API."
        except Exception as e:
            print(f"Error speech: {e}"); listening_status_message = "Error speech tidak diketahui."
        if not running: break
    print("Thread speech recognition dihentikan.")

running = True
game_won = False
clock = pygame.time.Clock()

if not maze_display_image or not maze_collision_mask:
    print("PERHATIAN: Labirin atau masker tabrakan tidak dimuat dengan benar.")

if microphone:
    listener_thread = threading.Thread(target=listen_for_hint_command, daemon=True)
    listener_thread.start()
else:
    listening_status_message = "Speech recognition tidak aktif (mikrofon error)."

cam_window_name = 'Kontrol Hidung - Kalibrasi & Kuadran'
last_animation_update_time = pygame.time.get_ticks()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if hint_requested_by_voice and not hint_active:
        print("Mengaktifkan hint...")
        if load_maze_assets(MAZE_IMAGE_PATH_HINT, False):
            hint_active = True
            print(f"Labirin hint dimuat untuk TAMPILAN. Posisi pemain TETAP.")
            listening_status_message = "Hint sudah aktif."
        else:
            print("Gagal memuat labirin hint.")
            listening_status_message = "Gagal load hint."
        hint_requested_by_voice = False

    dx_nose, dy_nose = 0, 0
    if cap.isOpened():
        success, frame = cap.read()
        if success:
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            cam_h, cam_w, _ = frame.shape
            results = face_mesh.process(frame_rgb)
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # --- PERBAIKAN MEDIAPIPE ---
                    nose_landmark = face_landmarks.landmark[1] # Landmark ke-1 adalah ujung hidung
                    current_nose_x = int(nose_landmark.x * cam_w)
                    current_nose_y = int(nose_landmark.y * cam_h)
                    # --- AKHIR PERBAIKAN MEDIAPIPE ---
                    cv2.circle(frame, (current_nose_x, current_nose_y), 5, (0, 255, 255), -1)
                    if not initial_nose_position_set:
                        initial_nose_x, initial_nose_y = current_nose_x, current_nose_y
                        initial_nose_position_set = True; print(f"Kalibrasi hidung di: ({initial_nose_x}, {initial_nose_y})")
                    if initial_nose_position_set:
                        cv2.rectangle(frame, (initial_nose_x - NEUTRAL_ZONE_RADIUS, initial_nose_y - NEUTRAL_ZONE_RADIUS), (initial_nose_x + NEUTRAL_ZONE_RADIUS, initial_nose_y + NEUTRAL_ZONE_RADIUS), GREEN_NEUTRAL, 2)
                        cv2.line(frame, (initial_nose_x, 0), (initial_nose_x, cam_h), (255,0,0),1); cv2.line(frame, (0, initial_nose_y), (cam_w, initial_nose_y), (255,0,0),1)
                        is_in_neutral = (initial_nose_x - NEUTRAL_ZONE_RADIUS < current_nose_x < initial_nose_x + NEUTRAL_ZONE_RADIUS and
                                         initial_nose_y - NEUTRAL_ZONE_RADIUS < current_nose_y < initial_nose_y + NEUTRAL_ZONE_RADIUS)
                        if is_in_neutral: dx_nose, dy_nose = 0,0
                        else:
                            if current_nose_x > initial_nose_x and current_nose_y < initial_nose_y: dx_nose, dy_nose = player_speed, -player_speed
                            elif current_nose_x < initial_nose_x and current_nose_y < initial_nose_y: dx_nose, dy_nose = -player_speed, -player_speed
                            elif current_nose_x < initial_nose_x and current_nose_y > initial_nose_y: dx_nose, dy_nose = -player_speed, player_speed
                            elif current_nose_x > initial_nose_x and current_nose_y > initial_nose_y: dx_nose, dy_nose = player_speed, player_speed
                    break
            else:
                if initial_nose_position_set: dx_nose, dy_nose = 0,0
                else: cv2.putText(frame, "Posisikan hidung", (cam_w//2-100, cam_h//2), cv2.FONT_HERSHEY_SIMPLEX, 0.8, WHITE, 2)
            cv2.imshow(cam_window_name, frame)

    if not game_won:
        original_player_x, original_player_y = player_rect.x, player_rect.y
        if dx_nose != 0:
            player_rect.x += dx_nose
            if maze_collision_mask and check_maze_collision_with_masks(player_rect, player_mask, maze_collision_mask, MAZE_OFFSET_X, MAZE_OFFSET_Y):
                player_rect.x = original_player_x
        if dy_nose != 0:
            player_rect.y += dy_nose
            if maze_collision_mask and check_maze_collision_with_masks(player_rect, player_mask, maze_collision_mask, MAZE_OFFSET_X, MAZE_OFFSET_Y):
                player_rect.y = original_player_y
        if maze_boundary_rect : player_rect.clamp_ip(maze_boundary_rect)
        if not player_has_moved_from_spawn:
            current_center_vec = pygame.math.Vector2(player_rect.centerx, player_rect.centery)
            initial_center_vec = pygame.math.Vector2(initial_player_center_x, initial_player_center_y)
            if current_center_vec.distance_to(initial_center_vec) > max(player_actual_width, player_actual_height) * 0.5:
                player_has_moved_from_spawn = True; print("Pemain bergerak dari spawn awal.")
        if player_has_moved_from_spawn and maze_boundary_rect:
            on_perimeter = (player_rect.left <= maze_boundary_rect.left or
                            player_rect.right >= maze_boundary_rect.right or
                            player_rect.top <= maze_boundary_rect.top or
                            player_rect.bottom >= maze_boundary_rect.bottom) and \
                           (player_rect.width > 0 and player_rect.height > 0)
            if on_perimeter:
                if player_rect.colliderect(maze_boundary_rect) and not maze_boundary_rect.contains(player_rect):
                     dist_vec = pygame.math.Vector2(player_rect.centerx, player_rect.centery).distance_to(pygame.math.Vector2(initial_player_center_x, initial_player_center_y))
                     if dist_vec > MIN_DISTANCE_FROM_SPAWN_FOR_WIN:
                        game_won = True; print(f"MENANG! Jarak dari spawn: {dist_vec:.1f}")

    if player_frames:
        current_ticks = pygame.time.get_ticks()
        if current_ticks - last_animation_update_time > ANIMATION_FRAME_DURATION:
            current_player_frame_index = (current_player_frame_index + 1) % len(player_frames)
            last_animation_update_time = current_ticks

    screen.fill(WHITE)
    pygame.draw.rect(screen, UI_BACKGROUND, (0, 0, WINDOW_WIDTH, MAZE_OFFSET_Y))
    pygame.draw.rect(screen, UI_BACKGROUND, (0, MAZE_OFFSET_Y + actual_maze_height, WINDOW_WIDTH, WINDOW_HEIGHT - (MAZE_OFFSET_Y + actual_maze_height)))
    pygame.draw.rect(screen, UI_BACKGROUND, (0, MAZE_OFFSET_Y, MAZE_OFFSET_X, actual_maze_height))
    pygame.draw.rect(screen, UI_BACKGROUND, (MAZE_OFFSET_X + actual_maze_width, MAZE_OFFSET_Y, WINDOW_WIDTH - (MAZE_OFFSET_X + actual_maze_width), actual_maze_height))
    score_text_surface = small_font.render("Skor: 0", True, BLACK)
    screen.blit(score_text_surface, (10, 10))
    speech_status_surf = micro_font.render(listening_status_message, True, YELLOW_HINT_MSG if "hint" in listening_status_message.lower() or "diminta" in listening_status_message.lower() else BLACK)
    screen.blit(speech_status_surf, (10, WINDOW_HEIGHT - 30))
    hint_btn_rect = pygame.Rect(WINDOW_WIDTH - 110, 10, 100, 30)
    pygame.draw.rect(screen, (0,200,0) if not hint_active else (100,100,100), hint_btn_rect)
    hint_txt_surf = small_font.render("Hint" if not hint_active else "Hint ON", True, BLACK)
    screen.blit(hint_txt_surf, hint_txt_surf.get_rect(center=hint_btn_rect.center))

    if maze_display_image:
        screen.blit(maze_display_image, (MAZE_OFFSET_X, MAZE_OFFSET_Y))
    else:
        if actual_maze_width > 0 and actual_maze_height > 0:
            placeholder_rect = pygame.Rect(MAZE_OFFSET_X, MAZE_OFFSET_Y, actual_maze_width, actual_maze_height)
            pygame.draw.rect(screen, BLACK, placeholder_rect, 2)
            text_surface = small_font.render("Error Labirin", True, BLACK)
            screen.blit(text_surface, (placeholder_rect.centerx - text_surface.get_width() // 2, placeholder_rect.centery - text_surface.get_height() // 2))

    if player_frames and player_mask:
        screen.blit(player_frames[current_player_frame_index], player_rect) # <--- PERBAIKAN ANIMASI
    else:
        pygame.draw.rect(screen, BLACK, player_rect)

    if game_won:
        win_text_surface = font.render("KAMU MENANG!", True, BLUE)
        win_text_rect = win_text_surface.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
        pygame.draw.rect(screen, WHITE, win_text_rect.inflate(20,20))
        screen.blit(win_text_surface, win_text_rect)

    pygame.display.flip()
    key_cv = cv2.waitKey(1)
    if key_cv & 0xFF == 27: running = False
    clock.tick(30)

if cap.isOpened(): cap.release()
cv2.destroyAllWindows()
pygame.quit()
print("Game ditutup.")
sys.exit()

SyntaxError: invalid syntax (3124163397.py, line 330)